In [ ]:
%load_ext autoreload
%autoreload 2

https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb

In [ ]:
from holo_table.utils.cv import cv_imshow
from holo_table.utils.mediapipe import get_default_hand_connections
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from pathlib import Path
from typing import cast
import cv2 as cv
import matplotlib.pyplot as plt
import mediapipe as mp
import mediapipe.python.solutions.drawing_styles as mp_drawing_styles
import mediapipe.python.solutions.drawing_utils as mp_drawing_utils
import mediapipe.python.solutions.hands as mp_hands
import numpy as np


In [ ]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 3
HANDEDNESS_TEXT_COLOR = (88, 205, 54)  # vibrant green


def draw_landmarks_on_image(rgb_image, detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    handedness_list = detection_result.handedness
    annotated_image = np.copy(rgb_image)

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]
        handedness = handedness_list[idx]

        # Draw the hand landmarks.
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend(
            [
                landmark_pb2.NormalizedLandmark(
                    x=landmark.x, y=landmark.y, z=landmark.z
                )
                for landmark in hand_landmarks
            ]
        )
        mp_drawing_utils.draw_landmarks(
            annotated_image,
            hand_landmarks_proto,
            get_default_hand_connections(),
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style(),
        )

        # Get the top left corner of the detected hand's bounding box.
        height, width, _ = annotated_image.shape
        x_coordinates = [landmark.x for landmark in hand_landmarks]
        y_coordinates = [landmark.y for landmark in hand_landmarks]
        text_x = int(min(x_coordinates) * width)
        text_y = int(min(y_coordinates) * height) - MARGIN

        # Draw handedness (left or right hand) on the image.
        cv.putText(
            annotated_image,
            f"{handedness[0].category_name}",
            (text_x, text_y),
            cv.FONT_HERSHEY_DUPLEX,
            FONT_SIZE,
            HANDEDNESS_TEXT_COLOR,
            FONT_THICKNESS,
            cv.LINE_AA,
        )

    return annotated_image


In [ ]:
mp_model_fol = Path("~/.mediapipe/models").expanduser()
hand_landmark_model_path = mp_model_fol / "hand_landmarker.task"
if not hand_landmark_model_path.exists():
    print(
        "Download the hand landmark model using\n"
        "curl "
        "https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/hand_landmarker.task"
        f" --create-dirs -o {hand_landmark_model_path}"
    )


In [ ]:
data_fol = Path("../data/sample").absolute().resolve()
sample_image_path = data_fol / "woman_hands.jpg"
if not sample_image_path.exists():
    print(
        "Download the sample image using\n"
        "curl "
        "https://storage.googleapis.com/mediapipe-tasks/hand_landmarker/woman_hands.jpg"
        f" --create-dirs -o {sample_image_path}"
    )


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks.python.core.base_options import BaseOptions
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarkerOptions
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarker

# from here
# https://github.com/google/mediapipe/blob/master/mediapipe/python/pybind/image.cc
# from mediapipe.python._framework_bindings
# ._framework_bindings.image import Image

# STEP 2: Create an ImageClassifier object.
base_options = BaseOptions(model_asset_path=str(hand_landmark_model_path))
options = HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = HandLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file(str(sample_image_path))

# STEP 4: Detect hand landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the classification result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv_imshow(cv.cvtColor(annotated_image, cv.COLOR_RGB2BGR))


In [ ]:
# # or import it like this but still no type hints
# from mediapipe.python import Image
# Image.create_from_file(str(sample_image_path))

In [ ]:
(
    image.width,
    image.height,
    image.channels,
    image.numpy_view().shape,
)
